# 4: Forward algorithm

In this notebook we will implement the forward algorithm together. 

## 1. Loop-based forward

We will first implement the algorithm using loops, and then explore vectorized implementations which are more efficient.

For this example, we won't train the HMM. We will simply use mad up values for initial, emission and transition probabilities.

Our `SimpleHMM` recognizes only three words `"I"`, `"am"` and `"Sam"`. It has three states (or tags) `"PRP"`, `"NN"` and `"VBN"`. 

Initial probabilities:

```
PRP   NN   VBN
0.9   0.05 0.05
```

Emission probabilities:

```
     PRP   NN    VBN
I    0.95  0.025 0.025
am   0.025 0.025 0.95
Sam  0.025 0.95  0.025
```

Transition probabilities:

```
     PRP    NN    VBN
PRP  0.05   0.05  0.9
NN   0.05   0.05  0.9
VBN  0.05   0.9   0.05
```

The forward algorithm takes a sentence as input and returns its total probability. We will need to:

1. Initialize a trellis (np.array of dimension |tag set size| x |sentence length|)
1. Initialize the first column of alpha values in the trellis
1. Iteratively fill all the rest of the columns in the trellis
1. Return the sum of the probabilities in the last column.

Remember that for the sentence "I am Sam":

$$\alpha_{2}(NN) = \sum_{t} \alpha_{1}(t) \cdot tr_{t,NN} \cdot em_{NN,Sam}$$

**Indexing in our code starts at 0, which is why $\alpha_2$ refers to the last (third) column in the trellis!**

In [1]:
import numpy as np

# Definition of vocabulary and tagset.
TAGS = ["PRP", "NN", "VBN"]
TAGCOUNT = 3
VOCAB = {"I":0, "am": 1, "Sam":2}

class SimpleHMM:
    def __init__(self):
        self.init_prob = np.array([0.9, 0.05, 0.05]) # 90% chance to start in the PRON state

        self.em_prob = np.array([[0.95,  0.025, 0.025],
                                 [0.025, 0.025, 0.95],
                                 [0.025, 0.95,  0.025]])

        self.tr_prob = np.array([[0.05, 0.05, 0.9],
                                 [0.05, 0.05, 0.9],
                                 [0.05, 0.9,  0.05]])

    def forward(self, s):
        s = [VOCAB[w] for w in s]
        
        trellis = np.zeros((len(TAGS), len(s)))

        # Fill in the first column of the trellis
        for t in range(TAGCOUNT):
            alpha_0_tag = self.init_prob[t] * self.em_prob[t,s[0]]
            trellis[t,0] = alpha_0_tag

        # Compute alpha n based on alpha n-1
        for i in range(1,len(s)):
            for t1 in range(TAGCOUNT):
                alpha_n_t1 = 0
                for t2 in range(TAGCOUNT):
                    alpha_n_1_t2 = trellis[t2, i-1]
                    tr_prob = self.tr_prob[t2, t1]
                    em_prob = self.em_prob[t1,s[i]] 
                    alpha_n_t1 += alpha_n_1_t2 * tr_prob * em_prob
                trellis[t1,i] = alpha_n_t1
        
        total_prob = 0
        for t in range(len(TAGS)):      # sum up the last column
            total_prob += trellis[t,len(s)-1]

        return total_prob

Let's initialize a `SimpleHMM` and compute the total probability of the sentences "I am Sam" and "am I Sam". 

In [2]:
hmm = SimpleHMM()

print(hmm.forward("I am Sam".split()))
print(hmm.forward("am I Sam".split()))

0.6279759394531248
0.0008285410156250001


Loops are actually a very slow way to process Numpy arrays. Using the predefined operations like addition and dot (or inner) product can substantially speed up your code. 

Let's try to add the values in a 10,000,000-dimensional first vector using a loop and then using the built-in `sum()` member function. We'll time the execution. 

In [3]:
%%time

a = np.ones((1,10000000))

tot = 0

for i in range(10000000):
    tot += 1
    
print(tot)

10000000
CPU times: user 1.07 s, sys: 45.7 ms, total: 1.11 s
Wall time: 1.17 s


In [4]:
%%time

a = np.ones((1,10000000))
print(a.sum())

10000000.0
CPU times: user 34 ms, sys: 47 ms, total: 80.9 ms
Wall time: 93.2 ms


## 2. Vectorizing the computation of $alpha_n(tag)$

We can speed the forward algorithm up quite a bit by noticing that all the `alpha_n` probabilities can be computed at once using the following formula:

$$\alpha_{2}(NN) = \alpha_1 \cdot tr_{:,NN} \cdot em_{NN, Sam}$$

$\alpha_1$ refers to the vector containing all the `alpha_1(tag)` probabilities. This is in fact column `1` of our trellis. `tr_{:,NN}` refers to the transition probabilites from any tag to the tag `NN` which is column 2 of our transition probability matrix (2 because the index of `NN` in `TAGS` is 2):

$$\alpha_{2}(NN) =\begin{bmatrix} \alpha_{1}(PRP) & \alpha_{1}(NN) & \alpha_{1}(VBN)\end{bmatrix} \cdot \begin{bmatrix}tr_{PRP,NN} \\ tr_{NN,NN} \\ tr_{VBN,NN}\end{bmatrix} \cdot em_{NN, Sam}$$

Notice that the code is much shrter than in the loop-based implementation.

Let's spend some time computing and making sure that this gives us the correct result.

In [5]:
class VectorizedHMM:
    def __init__(self):
        self.init_prob = np.array([0.9, 0.05, 0.05]) # 90% chance to start in the PRON state

        self.em_prob = np.array([[0.95,  0.025, 0.025],
                                 [0.025, 0.025, 0.95],
                                 [0.025, 0.95,  0.025]])

        self.tr_prob = np.array([[0.05, 0.05, 0.9],
                                 [0.05, 0.05, 0.9],
                                 [0.05, 0.9,  0.05]])

    def forward(self, s):
        s = [VOCAB[w] for w in s]
        
        trellis = np.zeros((len(TAGS), len(s)))
        
        trellis[:,0] = self.init_prob * self.em_prob[:,s[0]]

        for i in range(1,len(s)):
            alpha_n_1 = trellis[:,i-1]
            for t in range(TAGCOUNT):
                # @ refers to the inner or dot product of two arrays.
                alpha_n_t = (alpha_n_1 @ self.tr_prob[:,t]) * self.em_prob[t,s[i]]  
                # self.em_prob[t,s[i]]: given the word s[i], the probability of emitting the tag t
                trellis[t,i] = alpha_n_t

        return trellis[:,len(s)-1].sum()

Let's initialize a `VectorizedHMM` and compute the total probability of the sentences "I am Sam" and "am I Sam". 

In [6]:
hmm = VectorizedHMM()

print(hmm.forward("I am Sam".split()))
print(hmm.forward("am I Sam".split()))

0.6279759394531249
0.000828541015625


## 3. Vectorizing the computation of $\alpha_n$

We can actually optimize this even further by noticing that all the $\alpha_n(tag)$ probabilites can be computed at once. 

First, a bit of notation. The operator $\odot$ is the pointwise product of two matrices/vectors:

$$[x_1\ x_3\ x_3] \odot [y_1\ y_2\ y_3] = [x_1\cdot y_1\ x_2\cdot y_2\ x_3\cdot y_3]$$

For two numpy arrays `x` and `y`, the pointwise product is just `x * y`.

Now, we can compute the entire 3rd column of the trellis $\alpha_2$ in one step:

$$\alpha_{2} = \alpha_1 \cdot tr \cdot em_{:, Sam}$$

$$=\Big(\begin{bmatrix} \alpha_{1}(PRP) & \alpha_{1}(NN) & \alpha_{1}(VBN)\end{bmatrix} \begin{bmatrix}tr_{PRP,PRP} & tr_{PRP,NN} & tr_{PRP,VBN}\\ tr_{NN,PRP} & tr_{NN,NN} & tr_{NN,VBN}\\ tr_{VBN,PRP} & tr_{VBN,NN} & tr_{VBN,VBN}\end{bmatrix}\Big) \odot \begin{bmatrix}e_{PRP,Sam} & e_{NN,Sam} & e_{VBN,Sam}\end{bmatrix}$$

Let's spend some time computing and making sure that this gives us the correct result.

In [7]:
class FullyVectorizedHMM:
    def __init__(self):
        self.tags = {"PRP":0, "NN":1, "VBN":2}
        self.vocabulary = {"I":0, "am": 1, "Sam":2}
        
        self.init_prob = np.array([0.9, 0.05, 0.05]) # 90% chance to start in the PRON state

        self.em_prob = np.array([[0.95,  0.025, 0.025],
                                 [0.025, 0.025, 0.95],
                                 [0.025, 0.95,  0.025]])

        self.tr_prob = np.array([[0.05, 0.05, 0.9],
                                 [0.05, 0.05, 0.9],
                                 [0.05, 0.9,  0.05]])

    def forward(self, s):
        s = [self.vocabulary[w] for w in s]
        
        trellis = np.zeros((len(self.tags), len(s)))
        
        trellis[:,0] = self.init_prob * self.em_prob[:,s[0]]

        for i in range(1,len(s)):
            alpha_n_1 = trellis[:,i-1]
            # @ refers to the inner or dot product of two arrays. 
            # When the arrays are matrices, this is just regular matrix product.
            alpha_n = (alpha_n_1 @ self.tr_prob) * self.em_prob[:,s[i]]
            # this time self.em_prob[:,s[i]] gives all the column of a given tag
            trellis[:,i] = alpha_n   # assign the whole column to trellis

        return trellis[:,len(s)-1].sum()

Let's initialize a `FullyVectorizedHMM` and compute the total probability of the sentences "I am Sam" and "am I Sam". 

In [8]:
hmm = FullyVectorizedHMM()

print(hmm.forward("I am Sam".split()))
print(hmm.forward("am I Sam".split()))

0.6279759394531249
0.000828541015625


## 4. Using log-probabilities

Very often we want to convert all probabilities to log-probabilities in initial, emission and transition matrices.

We need to remember that product of probabilites $p\cdot q$ corresponds to addition of log-probabilities $\log p + \log q$

In [9]:
p = 0.5
q = 0.2

print(np.log2(p) + np.log2(q))
print(np.log2(p*q))

-3.321928094887362
-3.321928094887362


Going over to log-probabilities does not influence maximization which is what we'll need to do in the Viterbi algorithm: 

In [10]:
a = np.array([[0.1, 0.8],[0.7,0.2]])

print(a.max(axis=1))
print(a.argmax(axis=1))

[0.8 0.7]
[1 0]


In [11]:
b = np.log2(a)
print(b.max(axis=1))
print(b.argmax(axis=1))

[-0.32192809 -0.51457317]
[1 0]


In [12]:
print(np.exp2(b.max(axis=1)))

[0.8 0.7]
